In [3]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from align import AlignDlib  # Pastikan file align.py dan landmarks.dat tersedia di direktori yang sesuai


d:\anaconda3\envs\tf\lib\site-packages\h5py\__init__.py:36: UserWarning: h5py is running against HDF5 1.14.3 when it was built against 1.14.2, this may cause problems
  _warn(("h5py is running against HDF5 {0} when it was built against {1}, "


In [21]:
# Fungsi untuk memuat gambar dengan konversi BGR ke RGB
def load_image(path):
    img = cv2.imread(path, 1)
    return img[...,::-1]

# Inisialisasi AlignDlib untuk alignment
alignment = AlignDlib('models/landmarks.dat')

# Fungsi untuk load dan align gambar wajah
def load_and_align_images(images_folder, alignment):
    images = []
    labels = []
    for person_name in os.listdir(images_folder):
        person_folder = os.path.join(images_folder, person_name)
        if not os.path.isdir(person_folder):
            continue
        for image_name in os.listdir(person_folder):
            image_path = os.path.join(person_folder, image_name)
            img = load_image(image_path)
            bb = alignment.getLargestFaceBoundingBox(img)
            aligned_face = align_and_crop(img, bb)
            if aligned_face is not None:
                images.append(aligned_face)
                labels.append(person_name)
                # Menyimpan hasil align untuk diplot
                # plot_aligned_image(img, bb, aligned_face, person_name)
    return np.array(images), np.array(labels)

# Fungsi untuk align dan crop wajah tanpa warp
def align_and_crop(image, bb):
    # Mendapatkan koordinat bounding box
    left = bb.left()
    right = bb.right()
    top = bb.top()
    bottom = bb.bottom()
    
    # Memotong wajah dari gambar menggunakan bounding box
    face = image[top:bottom, left:right]
    
    # Rotasi wajah agar lebih lurus jika perlu
    # Misalnya, jika menggunakan angle of rotation sebesar 0.05 radian
    angle = 0.05
    M = cv2.getRotationMatrix2D((face.shape[1]//2, face.shape[0]//2), angle, 1)
    rotated_face = cv2.warpAffine(face, M, (face.shape[1], face.shape[0]))

    # Resize ke ukuran yang diinginkan (contoh: 150x150)
    aligned_face = cv2.resize(rotated_face, (150, 150))

    return aligned_face

# Fungsi untuk memplot hasil align
# def plot_aligned_image(image, bb, aligned_face, person_name):
#     # Membuat figure dan axis
#     fig, ax = plt.subplots(1, 2, figsize=(10, 5))

#     # Plot gambar asli dengan bounding box
#     ax[0].imshow(image)
#     ax[0].add_patch(patches.Rectangle((bb.left(), bb.top()), bb.width(), bb.height(), fill=False, color='red'))
#     ax[0].set_title('Original Image')

#     # Plot wajah yang sudah di-align
#     ax[1].imshow(aligned_face)
#     ax[1].set_title(f'Aligned Face - {person_name}')

#     # Menyembunyikan axis pada plot
#     for ax_single in ax:
#         ax_single.axis('off')

#     # Menampilkan plot
#     plt.tight_layout()
#     plt.show()

In [22]:
# Memuat dan align gambar-gambar dari dataset
dataset_folder = 'images'
images, labels = load_and_align_images(dataset_folder, alignment)

# Konversi label ke dalam bentuk one-hot encoding
label_to_index = {label: i for i, label in enumerate(np.unique(labels))}
index_to_label = {v: k for k, v in label_to_index.items()}
labels = np.array([label_to_index[label] for label in labels])

# Split dataset menjadi training dan testing set
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)


In [23]:
# Augmentasi data menggunakan ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

# Memuat data training dengan augmentasi
train_generator = train_datagen.flow(
    train_images, 
    train_labels,     
    batch_size=32,
)

# Memuat data validation tanpa augmentasi
validation_generator = validation_datagen.flow(
    test_images,                    
    test_labels, 
    batch_size=32,
)

In [8]:
# Augmentasi data menggunakan ImageDataGenerator
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Menggunakan flow() untuk menghasilkan data augmented dalam bentuk batch
augmented_images = []
augmented_labels = []

# Menambahkan data original ke augmented data
augmented_images.extend(train_images)
augmented_labels.extend(train_labels)

# Mendefinisikan berapa kali oversampling dilakukan (20 kali lipat)
oversampling_factor = 20

# Memproses setiap gambar untuk oversampling
for i in range(len(train_images)):
    img = train_images[i]
    label = train_labels[i]
    
    img = np.expand_dims(img, axis=0)
    label = np.expand_dims(label, axis=0)
    
    # Generate additional samples using datagen
    batches = 0
    for x_batch, y_batch in datagen.flow(img, label, batch_size=1):
        augmented_images.append(x_batch[0])
        augmented_labels.append(y_batch[0])
        batches += 1
        if batches >= oversampling_factor:
            break

# Konversi kembali ke np.array
train_images_augmented = np.array(augmented_images)
train_labels_augmented = np.array(augmented_labels)


In [20]:
# Membangun model CNN
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(512, activation='relu'),
    Dense(512, activation='relu'),
    Dense(512, activation='relu'),
    Dense(512, activation='relu'),
    Dense(len(label_to_index), activation='softmax')
])

# Kompilasi model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Melatih model dengan data yang di-augmentasi
history = model.fit(
    datagen.flow(train_images_augmented, train_labels_augmented, batch_size=32),
    epochs=200,
    steps_per_epoch=len(train_images_augmented) // 32,
    validation_data=(test_images, test_labels),
    verbose=1
)


Epoch 1/200
52/52 [==============================] - 8s 140ms/step - loss: 2.3000 - accuracy: 0.1025 - val_loss: 2.8056 - val_accuracy: 0.0000e+00
Epoch 2/200
52/52 [==============================] - 7s 139ms/step - loss: 2.2947 - accuracy: 0.1244 - val_loss: 2.7572 - val_accuracy: 0.0000e+00
Epoch 3/200
52/52 [==============================] - 7s 138ms/step - loss: 2.2925 - accuracy: 0.1262 - val_loss: 2.5761 - val_accuracy: 0.0000e+00
Epoch 4/200
50/52 [===========================>..] - ETA: 0s - loss: 2.2919 - accuracy: 0.1263

KeyboardInterrupt: 

In [24]:
# Membangun model CNN
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(512, activation='relu'),
    Dense(512, activation='relu'),
    Dense(512, activation='relu'),
    Dense(512, activation='relu'),
    Dense(len(label_to_index), activation='softmax')
])

In [25]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [27]:
history = model.fit(
    train_generator,
    epochs=500,
    validation_data=validation_generator,
    verbose=1
)

Epoch 1/500
3/3 [==============================] - 0s 51ms/step - loss: 2.2995 - accuracy: 0.1375
Epoch 2/500
3/3 [==============================] - 0s 111ms/step - loss: 2.2939 - accuracy: 0.1125
Epoch 3/500
3/3 [==============================] - 0s 111ms/step - loss: 2.2949 - accuracy: 0.1125
Epoch 4/500
3/3 [==============================] - 0s 113ms/step - loss: 2.2948 - accuracy: 0.1125
Epoch 5/500
3/3 [==============================] - 0s 157ms/step - loss: 2.2932 - accuracy: 0.1125
Epoch 6/500
3/3 [==============================] - 0s 106ms/step - loss: 2.2914 - accuracy: 0.1125
Epoch 7/500
3/3 [==============================] - 0s 114ms/step - loss: 2.2882 - accuracy: 0.1250
Epoch 8/500
3/3 [==============================] - 0s 112ms/step - loss: 2.2861 - accuracy: 0.1250
Epoch 9/500
3/3 [==============================] - 0s 103ms/step - loss: 2.2711 - accuracy: 0.1250
Epoch 10/500
3/3 [==============================] - 0s 105ms/step - loss: 2.2599 - accuracy: 0.1250
Epoch 11/5

In [28]:
loss, accuracy = model.evaluate(validation_generator)
print(f'Validation accuracy: {accuracy * 100:.2f}%')

1/1 [==============================] - 0s 285ms/step - loss: 4.7607 - accuracy: 0.2000
Validation accuracy: 20.00%
